<a href="https://colab.research.google.com/github/Viviankatheu/Future-Contraceptive-Use/blob/main/Intelligent_Forecasting_Challenge_Model_Future_Contraceptive__Use_in_Cote_d'Ivoire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intelligent Forecasting Challenge

## Defining The Question

### Specifying The Question

### Defining The Metric for Success

### Understanding The Context

### Recording The Experimental Design

### Data Relevance

## Reading The Data

## Checking The Data

## External Datasource Validation

## Tidying the datasets